# Part 1 - Identification of Squad Deficiencies

To determine what signings would best suit a club, we need to know where in the current squad the club is lacking in performance. Part One of this project involves identifying these deficiencies, based on how squad players compare with players of other clubs. 

In [1]:
import pandas as pd
import soccerdata as sd

[06/11/25 13:33:49] INFO     No custom team name replacements found. You can configure these in       ]8;id=588686;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=482604;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py#90\90]8;;\
                             /home/jasmine/soccerdata/config/teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=131002;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=861483;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_config.py#196\196]8;;\
                             /home/jasmine/soccerdata/config/league_dict.json.                                     

In [4]:
# 24/25 data
FBREF = sd.FBref(leagues="ENG-Premier League", seasons=2024)
FBREF

[06/11/25 13:38:48] INFO     Saving cached data to /home/jasmine/soccerdata/data/FBref               ]8;id=420546;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=439029;file:///home/jasmine/miniconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

The `soccerdata` package allows me to obtain season statistics for each player in the Premier League. I would like to group these players by their position, and select statistics (features) correspondingly for each position.

In [94]:
# grab all relevant statistics
player_stats = FBREF.read_player_season_stats(stat_type="standard")
player_defense = FBREF.read_player_season_stats(stat_type="defense")
player_passing = FBREF.read_player_season_stats(stat_type="passing")
player_shooting = FBREF.read_player_season_stats(stat_type="shooting")
player_keeper = FBREF.read_player_season_stats(stat_type="keeper")

In [5]:
player_stats.head()

nation    pos     age  born  \
                                                                              
league             season team    player                                      
ENG-Premier League 2425   Arsenal Ben White        ENG     DF  27-025  1997   
                                  Bukayo Saka      ENG  MF,FW  23-058  2001   
                                  David Raya       ESP     GK  29-048  1995   
                                  Declan Rice      ENG     MF  25-293  1999   
                                  Ethan Nwaneri    ENG  MF,FW  17-226  2007   

                                                Playing Time                   \
                                                          MP Starts  Min  90s   
league             season team    player                                        
ENG-Premier League 2425   Arsenal Ben White                7      6  585  6.5   
                                  Bukayo Saka              8      8  650  7.2   
                                  David Raya               9      9  810  9.0   
                                  Declan Rice              8      8  672  7.5   
                                  Ethan Nwaneri            4      0   27  0.3   

                                                Performance      ...  \
                                                        Gls Ast  ...   
league             season team    player                         ...   
ENG-Premier League 2425   Arsenal Ben White               0   1  ...   
                                  Bukayo Saka             3   7  ...   
                                  David Raya              0   0  ...   
                                  Declan Rice             0   1  ...   
                                  Ethan Nwaneri           0   0  ...   

                                                Per 90 Minutes              \
                                                           Gls   Ast   G+A   
league             season team    player                                     
ENG-Premier League 2425   Arsenal Ben White                0.0  0.15  0.15   
                                  Bukayo Saka             0.42  0.97  1.38   
                                  David Raya               0.0   0.0   0.0   
                                  Declan Rice              0.0  0.13  0.13   
                                  Ethan Nwaneri            0.0   0.0   0.0   

                                                                          \
                                                 G-PK G+A-PK    xG   xAG   
league             season team    player                                   
ENG-Premier League 2425   Arsenal Ben White       0.0   0.15  0.02  0.03   
                                  Bukayo Saka    0.42   1.38  0.31  0.49   
                                  David Raya      0.0    0.0   0.0   0.0   
                                  Declan Rice     0.0   0.13  0.05  0.17   
                                  Ethan Nwaneri   0.0    0.0  0.07  0.35   

                                                                       
                                                xG+xAG  npxG npxG+xAG  
league             season team    player                               
ENG-Premier League 2425   Arsenal Ben White       0.04  0.02     0.04  
                                  Bukayo Saka      0.8  0.31      0.8  
                                  David Raya       0.0   0.0      0.0  
                                  Declan Rice     0.22  0.05     0.22  
                                  Ethan Nwaneri   0.42  0.07     0.42  

[5 rows x 33 columns]

In [6]:
# unique positions
player_stats["pos"].unique()

<StringArray>
['DF', 'MF,FW', 'GK', 'MF', 'FW,MF', 'FW', 'MF,DF', 'FW,DF', 'DF,MF', 'DF,FW']
Length: 10, dtype: string

Note that some players play in more than one position. There is no way to partition the statistics based on which of the two positions they played, and so there are various approaches I can take here. 

1) Only take the first (primary) position
2) Take both positions (i.e., player will be part of two datasets), but under the pretense that it may skew the data
3) Instead of using simple Forward, Midfield, Defense positions, further split the positions into Attacking Midfielder and Defensive Midfielder

Option 3 seems the most reasonable. The order of the positions does matter though - e.g., "MF,DF" is not the same as "DF,MF." Additionally, some players are specified solely as "MF" even if they are Attacking or Defensive midfielders. **Transfermarkt** seems to have more specific player positions (as well as transfer-specific data, which I'll need later on), but this is only accessible via the R package `worldfootballR`.

In [12]:
player_stats[player_stats["pos"] == "MF"]

nation pos     age  \
                                                                           
league             season team        player                               
ENG-Premier League 2425   Arsenal     Declan Rice        ENG  MF  25-293   
                                      Jorginho           ITA  MF  32-318   
                                      Martin Ødegaard    NOR  MF  25-321   
                          Aston Villa Amadou Onana       BEL  MF  23-078   
                                      Boubacar Kamara    FRA  MF  24-345   
...                                                      ...  ..     ...   
                          Wolves      André              BRA  MF  23-109   
                                      Chiquinho          POR  MF  24-271   
                                      João Gomes         BRA  MF  23-264   
                                      Mario Lemina       GAB  MF  31-062   
                                      Tommy Doyle        ENG  MF  23-016   

                                                       born Playing Time  \
                                                                      MP   
league             season team        player                               
ENG-Premier League 2425   Arsenal     Declan Rice      1999            8   
                                      Jorginho         1991            2   
                                      Martin Ødegaard  1998            3   
                          Aston Villa Amadou Onana     2001            8   
                                      Boubacar Kamara  1999            1   
...                                                     ...          ...   
                          Wolves      André            2001            7   
                                      Chiquinho        2000            1   
                                      João Gomes       2001            9   
                                      Mario Lemina     1993            9   
                                      Tommy Doyle      2001            7   

                                                                        \
                                                      Starts  Min  90s   
league             season team        player                             
ENG-Premier League 2425   Arsenal     Declan Rice          8  672  7.5   
                                      Jorginho             2  150  1.7   
                                      Martin Ødegaard      3  253  2.8   
                          Aston Villa Amadou Onana         8  540  6.0   
                                      Boubacar Kamara      0    9  0.1   
...                                                      ...  ...  ...   
                          Wolves      André                5  363  4.0   
                                      Chiquinho            0    7  0.1   
                                      João Gomes           9  747  8.3   
                                      Mario Lemina         9  764  8.5   
                                      Tommy Doyle          1  204  2.3   

                                                      Performance      ...  \
                                                              Gls Ast  ...   
league             season team        player                           ...   
ENG-Premier League 2425   Arsenal     Declan Rice               0   1  ...   
                                      Jorginho                  0   0  ...   
                                      Martin Ødegaard           0   0  ...   
                          Aston Villa Amadou Onana              2   0  ...   
                                      Boubacar Kamara           0   0  ...   
...                                                           ...  ..  ...   
                          Wolves      André                     0   0  ...   
                                      Chiquinho                 0   0  ...   
                             

In [73]:
tm_player_stats = pd.read_csv("PL2425_player_stats.csv")
tm_player_stats

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
2,3,Manchester City,Premier League,England,Scott Carson,https://www.transfermarkt.com/scott-carson/pro...,Goalkeeper,39,England,26,0,0,0
3,4,Manchester City,Premier League,England,Spike Brits,https://www.transfermarkt.com/spike-brits/prof...,Goalkeeper,17,England,1,0,0,0
4,5,Manchester City,Premier League,England,Max Hudson,https://www.transfermarkt.com/max-hudson/profi...,Goalkeeper,17,Wales,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833
789,790,Ipswich Town,Premier League,England,Ali Al-Hamadi,https://www.transfermarkt.com/ali-al-hamadi/pr...,Centre-Forward,23,Iraq,15,13,1,276
790,791,Ipswich Town,Premier League,England,Wes Burns,https://www.transfermarkt.com/wes-burns/profil...,Right Winger,30,Wales,22,19,0,979


In [74]:
tm_player_stats['minutes_played'].describe()

count     792.000000
mean     1223.679293
std      1301.703849
min         0.000000
25%        17.750000
50%       775.500000
75%      2277.000000
max      4980.000000
Name: minutes_played, dtype: float64

In [75]:
tm_player_stats500 = tm_player_stats[tm_player_stats["minutes_played"] >= 500]
tm_player_stats500

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
5,6,Manchester City,Premier League,England,Josko Gvardiol,https://www.transfermarkt.com/josko-gvardiol/p...,Left-Back,23,Croatia,56,53,6,4513
6,7,Manchester City,Premier League,England,Rúben Dias,https://www.transfermarkt.com/ruben-dias/profi...,Centre-Back,28,Portugal,43,41,0,3215
7,8,Manchester City,Premier League,England,Rico Lewis,https://www.transfermarkt.com/rico-lewis/profi...,Right-Back,20,England,54,43,2,3072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,785,Ipswich Town,Premier League,England,Jack Clarke,https://www.transfermarkt.com/jack-clarke/prof...,Left Winger,24,England,39,36,3,1481
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Second Striker,28,England,29,23,2,944
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833


In [76]:
# the last 3 positions are outlying - see Rmd file
tm_player_stats500["player_pos"].unique()

array(['Goalkeeper', 'Left-Back', 'Centre-Back', 'Right-Back',
       'Central Midfield', 'Attacking Midfield', 'Centre-Forward',
       'Right Winger', 'Left Winger', 'Defensive Midfield',
       'Second Striker', 'Left Midfield', 'Right Midfield'], dtype=object)

In [77]:
# dorgu and j. ramsey - both left backs
tm_player_stats500[tm_player_stats500["player_pos"] == "Left Midfield"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
275,276,Manchester United,Premier League,England,Patrick Dorgu,https://www.transfermarkt.com/patrick-dorgu/pr...,Left Midfield,20,Denmark,20,20,0,1522
318,319,Aston Villa,Premier League,England,Jacob Ramsey,https://www.transfermarkt.com/jacob-ramsey/pro...,Left Midfield,24,England,46,45,4,2274


In [78]:
# rodrigo gomes is a right winger
tm_player_stats500[tm_player_stats500["player_pos"] == "Right Midfield"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
517,518,Wolverhampton Wanderers,Premier League,England,Rodrigo Gomes,https://www.transfermarkt.com/rodrigo-gomes/pr...,Right Midfield,21,Portugal,38,28,3,1068


In [79]:
tm_player_stats500[tm_player_stats500["player_pos"] == "Second Striker"]

,Unnamed: 0,team_name,league,country,player_name,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
126,127,Chelsea FC,Premier League,England,João Félix,https://www.transfermarkt.com/joao-felix/profi...,Second Striker,25,Portugal,28,20,7,948
527,528,Wolverhampton Wanderers,Premier League,England,Matheus Cunha,https://www.transfermarkt.com/matheus-cunha/pr...,Second Striker,26,Brazil,36,36,17,2832
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Second Striker,28,England,29,23,2,944


In [80]:
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Second Striker", "player_pos"] = "Centre-Forward"
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Left Midfield", "player_pos"] = "Left-Back"
tm_player_stats500.loc[tm_player_stats500["player_pos"] == "Right Midfield", "player_pos"] = "Right Winger"

In [81]:
tm_player_stats500["player_pos"].unique()

array(['Goalkeeper', 'Left-Back', 'Centre-Back', 'Right-Back',
       'Central Midfield', 'Attacking Midfield', 'Centre-Forward',
       'Right Winger', 'Left Winger', 'Defensive Midfield'], dtype=object)

I now need to merge the two player stats dataframes that I have. 

In [98]:
# drop the multilevel index
player_stats = player_stats.reset_index()
player_defense = player_defense.reset_index()
player_passing = player_passing.reset_index()
player_shooting = player_shooting.reset_index()
player_keeper = player_keeper.reset_index()
player_stats

league season     team           player nation    pos  \
                                                                         
0    ENG-Premier League   2425  Arsenal        Ben White    ENG     DF   
1    ENG-Premier League   2425  Arsenal      Bukayo Saka    ENG  MF,FW   
2    ENG-Premier League   2425  Arsenal       David Raya    ESP     GK   
3    ENG-Premier League   2425  Arsenal      Declan Rice    ENG     MF   
4    ENG-Premier League   2425  Arsenal    Ethan Nwaneri    ENG  MF,FW   
..                  ...    ...      ...              ...    ...    ...   
447  ENG-Premier League   2425   Wolves    Sam Johnstone    ENG     GK   
448  ENG-Premier League   2425   Wolves   Santiago Bueno    URU     DF   
449  ENG-Premier League   2425   Wolves      Tommy Doyle    ENG     MF   
450  ENG-Premier League   2425   Wolves       Toti Gomes    POR     DF   
451  ENG-Premier League   2425   Wolves  Yerson Mosquera    COL     DF   

        age  born Playing Time         ... Per 90 Minutes                    \
                            MP Starts  ...            Gls   Ast   G+A  G-PK   
0    27-025  1997            7      6  ...            0.0  0.15  0.15   0.0   
1    23-058  2001            8      8  ...           0.42  0.97  1.38  0.42   
2    29-048  1995            9      9  ...            0.0   0.0   0.0   0.0   
3    25-293  1999            8      8  ...            0.0  0.13  0.13   0.0   
4    17-226  2007            4      0  ...            0.0   0.0   0.0   0.0   
..      ...   ...          ...    ...  ...            ...   ...   ...   ...   
447  31-222  1993            5      5  ...            0.0   0.0   0.0   0.0   
448  25-359  1998            4      3  ...            0.0   0.0   0.0   0.0   
449  23-016  2001            7      1  ...            0.0  0.44  0.44   0.0   
450  25-291  1999            7      7  ...            0.0  0.15  0.15   0.0   
451  23-184  2001            5      5  ...            0.0   0.0   0.0   0.0   

                                              
    G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0     0.15  0.02  0.03   0.04  0.02     0.04  
1     1.38  0.31  0.49    0.8  0.31      0.8  
2      0.0   0.0   0.0    0.0   0.0      0.0  
3     0.13  0.05  0.17   0.22  0.05     0.22  
4      0.0  0.07  0.35   0.42  0.07     0.42  
..     ...   ...   ...    ...   ...      ...  
447    0.0   0.0  0.01   0.01   0.0     0.01  
448    0.0  0.07   0.0   0.07  0.07     0.07  
449   0.44  0.12  0.04   0.16  0.12     0.16  
450   0.15   0.0  0.08   0.08   0.0     0.08  
451    0.0  0.07  0.02   0.09  0.07     0.09  

[452 rows x 37 columns]

In [99]:
player_stats500 = player_stats[player_stats["Playing Time"]['Min'] > 500]
player_stats500

league season     team             player nation    pos  \
                                                                           
0    ENG-Premier League   2425  Arsenal          Ben White    ENG     DF   
1    ENG-Premier League   2425  Arsenal        Bukayo Saka    ENG  MF,FW   
2    ENG-Premier League   2425  Arsenal         David Raya    ESP     GK   
3    ENG-Premier League   2425  Arsenal        Declan Rice    ENG     MF   
6    ENG-Premier League   2425  Arsenal  Gabriel Magalhães    BRA     DF   
..                  ...    ...      ...                ...    ...    ...   
440  ENG-Premier League   2425   Wolves       Mario Lemina    GAB     MF   
441  ENG-Premier League   2425   Wolves      Matheus Cunha    BRA  MF,FW   
443  ENG-Premier League   2425   Wolves      Nélson Semedo    POR     DF   
445  ENG-Premier League   2425   Wolves    Rayan Aït-Nouri    ALG     DF   
450  ENG-Premier League   2425   Wolves         Toti Gomes    POR     DF   

        age  born Playing Time         ... Per 90 Minutes                    \
                            MP Starts  ...            Gls   Ast   G+A  G-PK   
0    27-025  1997            7      6  ...            0.0  0.15  0.15   0.0   
1    23-058  2001            8      8  ...           0.42  0.97  1.38  0.42   
2    29-048  1995            9      9  ...            0.0   0.0   0.0   0.0   
3    25-293  1999            8      8  ...            0.0  0.13  0.13   0.0   
6    26-319  1997            9      9  ...           0.23   0.0  0.23  0.23   
..      ...   ...          ...    ...  ...            ...   ...   ...   ...   
440  31-062  1993            9      9  ...           0.12  0.12  0.24  0.12   
441  25-159  1999            9      8  ...           0.51   0.0  0.51  0.51   
443  30-352  1993            7      7  ...            0.0   0.3   0.3   0.0   
445  23-149  2001            9      9  ...           0.36  0.24  0.61  0.36   
450  25-291  1999            7      7  ...            0.0  0.15  0.15   0.0   

                                              
    G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0     0.15  0.02  0.03   0.04  0.02     0.04  
1     1.38  0.31  0.49    0.8  0.31      0.8  
2      0.0   0.0   0.0    0.0   0.0      0.0  
3     0.13  0.05  0.17   0.22  0.05     0.22  
6     0.23  0.17  0.03    0.2  0.17      0.2  
..     ...   ...   ...    ...   ...      ...  
440   0.24  0.12  0.03   0.15  0.12     0.15  
441   0.51  0.28  0.23   0.51  0.28     0.51  
443    0.3  0.05  0.15   0.19  0.05     0.19  
445   0.61  0.13  0.12   0.25  0.13     0.25  
450   0.15   0.0  0.08   0.08   0.0     0.08  

[183 rows x 37 columns]

In [82]:
len(set(player_stats500['player']) & set(tm_player_stats500['player_name']))

171

In [83]:
# players unaccounted for in the primary dataset
set(player_stats500['player']) - set(tm_player_stats500['player_name'])

{'Emerson Palmieri',
 'Idrissa Gana Gueye',
 'Illia Zabarnyi',
 'Joško Gvardiol',
 'Jurriën Timber',
 'Mateo Kovačić',
 'Max Kilman',
 'Son Heung-min',
 'Tomáš Souček',
 'Toti Gomes',
 'Valentino Livramento',
 'Victor Bernth Kristiansen'}

In [84]:
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Emerson", "player_name"] = "Emerson Palmieri"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Idrissa Gueye", "player_name"] = "Idrissa Gana Gueye"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Ilya Zabarnyi", "player_name"] = "Illia Zabarnyi"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Josko Gvardiol", "player_name"] = "Joško Gvardiol"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Jurrien Timber", "player_name"] = "Jurriën Timber"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Mateo Kovacic", "player_name"] = "Mateo Kovačić"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Maximilian Kilman", "player_name"] = "Max Kilman"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Heung-min Son", "player_name"] = "Son Heung-min"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Tomas Soucek", "player_name"] = "Tomáš Souček"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Toti", "player_name"] = "Toti Gomes"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Tino Livramento", "player_name"] = "Valentino Livramento"
tm_player_stats500.loc[tm_player_stats500["player_name"] == "Victor Kristiansen", "player_name"] = "Victor Bernth Kristiansen"

In [85]:
set(player_stats500['player']) - set(tm_player_stats500['player_name'])

set()

In [87]:
tm_player_stats500 = tm_player_stats500.rename(columns={"player_name": "player"})
tm_player_stats500

,Unnamed: 0,team_name,league,country,player,player_url,player_pos,player_age,nationality,in_squad,appearances,goals,minutes_played
0,1,Manchester City,Premier League,England,Ederson,https://www.transfermarkt.com/ederson/profil/s...,Goalkeeper,31,Brazil,48,37,0,3311
1,2,Manchester City,Premier League,England,Stefan Ortega,https://www.transfermarkt.com/stefan-ortega/pr...,Goalkeeper,32,Germany,57,21,0,1819
5,6,Manchester City,Premier League,England,Joško Gvardiol,https://www.transfermarkt.com/josko-gvardiol/p...,Left-Back,23,Croatia,56,53,6,4513
6,7,Manchester City,Premier League,England,Rúben Dias,https://www.transfermarkt.com/ruben-dias/profi...,Centre-Back,28,Portugal,43,41,0,3215
7,8,Manchester City,Premier League,England,Rico Lewis,https://www.transfermarkt.com/rico-lewis/profi...,Right-Back,20,England,54,43,2,3072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,785,Ipswich Town,Premier League,England,Jack Clarke,https://www.transfermarkt.com/jack-clarke/prof...,Left Winger,24,England,39,36,3,1481
786,787,Ipswich Town,Premier League,England,Conor Chaplin,https://www.transfermarkt.com/conor-chaplin/pr...,Centre-Forward,28,England,29,23,2,944
787,788,Ipswich Town,Premier League,England,Nathan Broadhead,https://www.transfermarkt.com/nathan-broadhead...,Left Winger,27,Wales,28,21,2,836
788,789,Ipswich Town,Premier League,England,George Hirst,https://www.transfermarkt.com/george-hirst/pro...,Centre-Forward,26,Scotland,29,29,5,833


In [103]:
player_stats500.columns.get_level_values(1)

Index(['', '', '', '', '', '', '', '', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls', 'Ast', 'G+A', 'G-PK',
       'G+A-PK', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG'],
      dtype='object')

In [104]:
player_stats500.columns = player_stats.columns.get_level_values(1)
player_stats500

,,,,,,,,,MP,Starts,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.0,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.42,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.0,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22
6,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.23,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15
441,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.51,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51
443,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.0,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19
445,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.36,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25


In [105]:
player_stats500.columns.values[0] = "league"
player_stats500.columns.values[1] = "season"
player_stats500.columns.values[2] = "team"
player_stats500.columns.values[3] = "player"
player_stats500.columns.values[4] = "nation"
player_stats500.columns.values[5] = "pos"
player_stats500.columns.values[6] = "age"
player_stats500.columns.values[7] = "born"
player_stats500

,league,season,team,player,nation,pos,age,born,MP,Starts,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.0,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.42,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.0,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22
6,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.23,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15
441,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.51,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51
443,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.0,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19
445,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.36,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25


In [106]:
full_player_stats = player_stats500.merge(tm_player_stats500[["player", "player_pos"]], how="left", on="player")
full_player_stats

,league,season,team,player,nation,pos,age,born,MP,Starts,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,player_pos
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,6,...,0.15,0.15,0.0,0.15,0.02,0.03,0.04,0.02,0.04,Right-Back
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,8,...,0.97,1.38,0.42,1.38,0.31,0.49,0.8,0.31,0.8,Right Winger
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goalkeeper
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,8,...,0.13,0.13,0.0,0.13,0.05,0.17,0.22,0.05,0.22,Central Midfield
4,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,9,...,0.0,0.23,0.23,0.23,0.17,0.03,0.2,0.17,0.2,Centre-Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,ENG-Premier League,2425,Wolves,Mario Lemina,GAB,MF,31-062,1993,9,9,...,0.12,0.24,0.12,0.24,0.12,0.03,0.15,0.12,0.15,Defensive Midfield
180,ENG-Premier League,2425,Wolves,Matheus Cunha,BRA,"MF,FW",25-159,1999,9,8,...,0.0,0.51,0.51,0.51,0.28,0.23,0.51,0.28,0.51,Centre-Forward
181,ENG-Premier League,2425,Wolves,Nélson Semedo,POR,DF,30-352,1993,7,7,...,0.3,0.3,0.0,0.3,0.05,0.15,0.19,0.05,0.19,Right-Back
182,ENG-Premier League,2425,Wolves,Rayan Aït-Nouri,ALG,DF,23-149,2001,9,9,...,0.24,0.61,0.36,0.61,0.13,0.12,0.25,0.13,0.25,Left-Back


In [107]:
full_player_stats.to_csv("PL2425_500min_player_stats.csv")

In [2]:
full_player_stats = pd.read_csv("PL2425_500min_player_stats.csv")
full_player_stats.head()

,Unnamed: 0,league,season,team,player,nation,pos,age,born,MP,...,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,player_pos
0,0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,27-025,1997,7,...,0.15,0.15,0.00,0.15,0.02,0.03,0.04,0.02,0.04,Right-Back
1,1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"MF,FW",23-058,2001,8,...,0.97,1.38,0.42,1.38,0.31,0.49,0.80,0.31,0.80,Right Winger
2,2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,29-048,1995,9,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Goalkeeper
3,3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25-293,1999,8,...,0.13,0.13,0.00,0.13,0.05,0.17,0.22,0.05,0.22,Central Midfield
4,4,ENG-Premier League,2425,Arsenal,Gabriel Magalhães,BRA,DF,26-319,1997,9,...,0.00,0.23,0.23,0.23,0.17,0.03,0.20,0.17,0.20,Centre-Back


In [3]:
full_player_stats["player_pos"].unique()

array(['Right-Back', 'Right Winger', 'Goalkeeper', 'Central Midfield',
       'Centre-Back', 'Left Winger', 'Centre-Forward',
       'Defensive Midfield', 'Left-Back', 'Attacking Midfield'],
      dtype=object)

The CSV file I now have contains the 184 players in the PL 24/25 season who played over 500 minutes for their club. I now want to split the dataset by position, so that I can compare Spurs players to some benchmark (either the league average for the position, or the top-10 average for the position). For each position, I need to obtain the relevant statistics.  

* Goalkeeper: "keeper" and "keeper_adv"
* Centre-Back, Left-Back, Right-Back: "defense" and "passing"
* Defensive Midfield: "defense" and "passing"
* Central Midfield: "passing" and "defense"
* Attacking Midfield: "passing", "goal_shot_creation", and "shooting"
* Left Winger, Right Winger: "passing", "goal_shot_creation", and "shooting"
* Centre-Forward: "goal_shot_creation" and "shooting"

For each position's dataset, I have to filter for the players of that position from the `full_player_stats` dataset, and grab the stats I desire.

### Goalkeepers

Specific statistics: 

* `GA90`: goals against per 90
* `Saves90`: [computed] number of saves per 90
* `Save%`: percentage of shots saved; (total shots on target against - goals scored against) / (total shots on target against), although note that not all shots on target are saved by the keeper; some are blocked by defenders
* `PSxG-GA/90`: post-shot expected goals minus goals allowed per 90; positive values indicate an above-average shot-stopping ability. Looks at the likelihood of the goalkeeper in question saving a shot based on the PSxG of where the shot ended up - [explanation here.](https://www.footballcritic.com/features/post-shot-expected-goals-what-is-it-and-why-is-it-different-from-expected-goalsc/773)
* `Cmp%`: completion percentage of passes longer than 40 yards; assesses distribution
* `#OPA90`: defensive actions outside the penalty box per 90; assesses "sweeper keeper" abilities - a keeper's ability to rush out of their goal and involve themselves in passing sequences higher up the pitch - [explanation here.](https://themastermindsite.com/tag/defensive-actions-outside-the-penalty-area/)

I'd like to note that I am excluding Clean Sheet percentages because of its large dependence on the team's defensive prowess as a whole, and I am also excluding Penalty Save percentages because penalties are relatively rare events that are generally unpredictable and are not thoroughly indicative of a keeper's abilities.

In [21]:
fbref_keeper = FBREF.read_player_season_stats(stat_type="keeper")
fbref_keeper_adv = FBREF.read_player_season_stats(stat_type="keeper_adv")

In [22]:
def flatten_df(df): 
    df = df.reset_index()
    df.columns = df.columns.get_level_values(1)
    df.columns.values[0] = "league"
    df.columns.values[1] = "season"
    df.columns.values[2] = "team"
    df.columns.values[3] = "player"
    df.columns.values[4] = "nation"
    df.columns.values[5] = "pos"
    df.columns.values[6] = "age"
    df.columns.values[7] = "born"
    
    return df

In [23]:
fbref_keeper = flatten_df(fbref_keeper)
fbref_keeper

,league,season,team,player,nation,pos,age,born,MP,Starts,...,W,D,L,CS,CS%,PKatt,PKA,PKsv,PKm,Save%
0,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38,38,...,20,14,4,13,34.2,3,3,0,0,0.0
1,ENG-Premier League,2425,Aston Villa,Emiliano Martínez,ARG,GK,31,1992,37,37,...,18,9,10,8,21.6,2,1,1,0,50.0
2,ENG-Premier League,2425,Aston Villa,Robin Olsen,SWE,GK,34,1990,4,1,...,1,0,0,1,100.0,1,1,0,0,0.0
3,ENG-Premier League,2425,Bournemouth,Kepa Arrizabalaga,ESP,GK,29,1994,31,31,...,13,7,11,8,25.8,4,4,0,0,0.0
4,ENG-Premier League,2425,Bournemouth,Mark Travers,IRL,GK,25,1999,5,5,...,2,2,1,1,20.0,0,0,0,0,<NA>
5,ENG-Premier League,2425,Bournemouth,Neto,BRA,GK,35,1989,2,2,...,0,2,0,0,0.0,0,0,0,0,<NA>
6,ENG-Premier League,2425,Brentford,Hákon Rafn Valdimarsson,ISL,GK,22,2001,2,1,...,0,0,1,1,100.0,0,0,0,0,<NA>
7,ENG-Premier League,2425,Brentford,Mark Flekken,NED,GK,31,1993,37,37,...,16,8,13,7,18.9,1,1,0,0,0.0
8,ENG-Premier League,2425,Brighton,Bart Verbruggen,NED,GK,21,2002,36,36,...,14,13,9,7,19.4,9,9,0,0,0.0
9,ENG-Premier League,2425,Brighton,Jason Steele,ENG,GK,33,1990,2,2,...,2,0,0,1,50.0,0,0,0,0,<NA>


In [24]:
fbref_keeper_adv = flatten_df(fbref_keeper_adv)
fbref_keeper_adv

,league,season,team,player,nation,pos,age,born,,GA,...,AvgLen,Att,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38.0,34,...,29.9,147,68.0,49.1,402,53,13.2,66,1.74,17.2
1,ENG-Premier League,2425,Aston Villa,Emiliano Martínez,ARG,GK,31,1992,35.5,45,...,30.4,172,59.3,46.3,498,56,11.2,35,0.99,13.7
2,ENG-Premier League,2425,Aston Villa,Robin Olsen,SWE,GK,34,1990,2.5,6,...,35.9,26,84.6,60.0,40,3,7.5,0,0.0,8.6
3,ENG-Premier League,2425,Bournemouth,Kepa Arrizabalaga,ESP,GK,29,1994,31.0,39,...,32.9,192,46.4,38.7,367,22,6.0,54,1.74,16.0
4,ENG-Premier League,2425,Bournemouth,Mark Travers,IRL,GK,25,1999,5.0,5,...,36.3,39,76.9,55.8,63,1,1.6,5,1.0,12.9
5,ENG-Premier League,2425,Bournemouth,Neto,BRA,GK,35,1989,2.0,2,...,36.8,15,46.7,38.9,30,2,6.7,3,1.5,12.6
6,ENG-Premier League,2425,Brentford,Hákon Rafn Valdimarsson,ISL,GK,22,2001,1.6,2,...,34.9,16,93.8,59.3,16,2,12.5,0,0.0,4.7
7,ENG-Premier League,2425,Brentford,Mark Flekken,NED,GK,31,1993,36.4,55,...,34.4,212,72.6,54.8,581,46,7.9,34,0.93,11.5
8,ENG-Premier League,2425,Brighton,Bart Verbruggen,NED,GK,21,2002,36.0,58,...,29.7,83,67.5,48.5,414,27,6.5,47,1.31,13.9
9,ENG-Premier League,2425,Brighton,Jason Steele,ENG,GK,33,1990,2.0,1,...,25.1,2,50.0,43.0,22,2,9.1,3,1.5,15.4


In [25]:
all_keeper_data = fbref_keeper.merge(fbref_keeper_adv, how="left", on="player")
all_keeper_data

,league_x,season_x,team_x,player,nation_x,pos_x,age_x,born_x,MP,Starts,...,AvgLen,Att,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38,38,...,29.9,147,68.0,49.1,402,53,13.2,66,1.74,17.2
1,ENG-Premier League,2425,Aston Villa,Emiliano Martínez,ARG,GK,31,1992,37,37,...,30.4,172,59.3,46.3,498,56,11.2,35,0.99,13.7
2,ENG-Premier League,2425,Aston Villa,Robin Olsen,SWE,GK,34,1990,4,1,...,35.9,26,84.6,60.0,40,3,7.5,0,0.0,8.6
3,ENG-Premier League,2425,Bournemouth,Kepa Arrizabalaga,ESP,GK,29,1994,31,31,...,32.9,192,46.4,38.7,367,22,6.0,54,1.74,16.0
4,ENG-Premier League,2425,Bournemouth,Mark Travers,IRL,GK,25,1999,5,5,...,36.3,39,76.9,55.8,63,1,1.6,5,1.0,12.9
5,ENG-Premier League,2425,Bournemouth,Neto,BRA,GK,35,1989,2,2,...,36.8,15,46.7,38.9,30,2,6.7,3,1.5,12.6
6,ENG-Premier League,2425,Brentford,Hákon Rafn Valdimarsson,ISL,GK,22,2001,2,1,...,34.9,16,93.8,59.3,16,2,12.5,0,0.0,4.7
7,ENG-Premier League,2425,Brentford,Mark Flekken,NED,GK,31,1993,37,37,...,34.4,212,72.6,54.8,581,46,7.9,34,0.93,11.5
8,ENG-Premier League,2425,Brighton,Bart Verbruggen,NED,GK,21,2002,36,36,...,29.7,83,67.5,48.5,414,27,6.5,47,1.31,13.9
9,ENG-Premier League,2425,Brighton,Jason Steele,ENG,GK,33,1990,2,2,...,25.1,2,50.0,43.0,22,2,9.1,3,1.5,15.4


In [30]:
all_keeper_data.columns

Index(['league_x', 'season_x', 'team_x', 'player', 'nation_x', 'pos_x',
       'age_x', 'born_x', 'MP', 'Starts', 'Min', '90s', 'GA_x', 'GA90', 'SoTA',
       'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%', 'PKatt', 'PKA_x', 'PKsv',
       'PKm', 'Save%', 'league_y', 'season_y', 'team_y', 'nation_y', 'pos_y',
       'age_y', 'born_y', '', 'GA_y', 'PKA_y', 'FK', 'CK', 'OG', 'PSxG',
       'PSxG/SoT', 'PSxG+/-', '/90', 'Cmp', 'Att', 'Cmp%', 'Att (GK)', 'Thr',
       'Launch%', 'AvgLen', 'Att', 'Launch%', 'AvgLen', 'Opp', 'Stp', 'Stp%',
       '#OPA', '#OPA/90', 'AvgDist'],
      dtype='object')

In [29]:
def filter_position_df(df, position, cols_to_keep):
    # only keep players present in 500min dataset
    df = df[df["player"].isin(list(full_player_stats['player']))]    
    # merge with 500min dataset to get full position name
    df = df.merge(full_player_stats[["player", "player_pos"]], how="left", on="player")
    # only keep players of given position
    df = df[df["player_pos"] == position]
    df = df[cols_to_keep]

    return df

In [32]:
filter_position_df(all_keeper_data, "Goalkeeper", ["team_x", "player", "nation_x", 
                                                   "player_pos", "Min", "GA90", "Saves", 
                                                   "Save%", '/90', "Cmp%", "#OPA/90"])

,team_x,player,nation_x,player_pos,Min,GA90,Saves,Save%,Save%,/90,Cmp%,#OPA/90
0,Arsenal,David Raya,ESP,Goalkeeper,3420,0.89,86,74.2,0.0,0.03,31.2,1.74
1,Aston Villa,Emiliano Martínez,ARG,Goalkeeper,3194,1.27,96,69.0,50.0,0.02,38.6,0.99
2,Brentford,Mark Flekken,NED,Goalkeeper,3275,1.51,150,73.4,0.0,-0.01,37.8,0.93
3,Brighton,Bart Verbruggen,NED,Goalkeeper,3240,1.61,87,65.7,0.0,-0.14,31.6,1.31
4,Chelsea,Robert Sánchez,ESP,Goalkeeper,2880,1.06,92,76.4,20.0,0.05,23.1,1.78
5,Crystal Palace,Dean Henderson,ENG,Goalkeeper,3420,1.34,101,66.7,50.0,0.08,26.2,1.05
6,Everton,Jordan Pickford,ENG,Goalkeeper,3420,1.16,117,73.0,100.0,0.13,37.7,1.5
7,Fulham,Bernd Leno,GER,Goalkeeper,3420,1.42,106,67.9,25.0,-0.08,38.4,0.87
8,Ipswich Town,Arijanet Muric,KVX,Goalkeeper,1620,1.83,67,70.0,0.0,-0.09,33.2,1.11
9,Leicester City,Mads Hermansen,DEN,Goalkeeper,2385,2.19,99,64.5,25.0,0.05,24.2,0.98


In [11]:
FBREF.read_player_season_stats(stat_type="shooting")

nation    pos age  born  \
                                                                            
league             season team    player                                    
ENG-Premier League 2425   Arsenal Ben White          ENG     DF  26  1997   
                                  Bukayo Saka        ENG  FW,MF  22  2001   
                                  David Raya         ESP     GK  28  1995   
                                  Declan Rice        ENG     MF  25  1999   
                                  Ethan Nwaneri      ENG  FW,MF  17  2007   
...                                                  ...    ...  ..   ...   
                          Wolves  Santiago Bueno     URU     DF  25  1998   
                                  Tom King           WAL     GK  29  1995   
                                  Tommy Doyle        ENG     MF  22  2001   
                                  Toti Gomes         POR     DF  25  1999   
                                  Yerson Mosquera    COL     DF  23  2001   

                                                    90s Standard          \
                                                             Gls  Sh SoT   
league             season team    player                                   
ENG-Premier League 2425   Arsenal Ben White        13.3        0   9   2   
                                  Bukayo Saka      19.3        6  66  22   
                                  David Raya       38.0        0   0   0   
                                  Declan Rice      31.5        4  48  14   
                                  Ethan Nwaneri    10.0        4  24   9   
...                                                 ...      ...  ..  ..   
                          Wolves  Santiago Bueno   18.8        0   6   3   
                                  Tom King          0.0        0   0   0   
                                  Tommy Doyle       5.4        0   9   2   
                                  Toti Gomes       29.1        0   7   2   
                                  Yerson Mosquera   4.9        0   2   0   

                                                               ...        \
                                                   SoT% Sh/90  ... G/SoT   
league             season team    player                       ...         
ENG-Premier League 2425   Arsenal Ben White        22.2  0.67  ...   0.0   
                                  Bukayo Saka      33.3  3.41  ...  0.23   
                                  David Raya       <NA>   0.0  ...  <NA>   
                                  Declan Rice      29.2  1.52  ...  0.29   
                                  Ethan Nwaneri    37.5  2.39  ...  0.44   
...                                                 ...   ...  ...   ...   
                          Wolves  Santiago Bueno   50.0  0.32  ...   0.0   
                                  Tom King         <NA>   0.0  ...  <NA>   
                                  Tommy Doyle      22.2  1.65  ...   0.0   
                                  Toti Gomes       28.6  0.24  ...   0.0   
                                  Yerson Mosquera   0.0  0.41  ...  <NA>   

                                                                    Expected  \
                                                   Dist FK PK PKatt       xG   
league             season team    player                                       
ENG-Premier League 2425   Arsenal Ben White        19.4  0  0     0      0.5   
                                  Bukayo Saka      15.6  1  1     1      6.8   
                                  David Raya       <NA>  0  0     0      0.0   
                                  Declan Rice      19.6  2  0     0      3.5   
                                  Ethan Nwaneri    18.5  1  0     0      1.2   
...                                                 ... .. ..   ...      ...   
                          Wolves  Santiago Bueno   14.0  0  0     0      0.4   
                                  Tom Kin